In [3]:
import boto3
dynamodb = boto3.resource('dynamodb', region_name = 'us-east-1')

# Create the DynamoDB table.
table = dynamodb.create_table(
    TableName='users',
    KeySchema=[
        {
            'AttributeName': 'username',
            'KeyType': 'HASH'
        },
        {
            'AttributeName': 'last_name',
            'KeyType': 'RANGE'
        }
    ],
    AttributeDefinitions=[
        {
            'AttributeName': 'username',
            'AttributeType': 'S'
        },
        {
            'AttributeName': 'last_name',
            'AttributeType': 'S'
        },
    ],
    ProvisionedThroughput={
        'ReadCapacityUnits': 5,
        'WriteCapacityUnits': 5
    }
)

# Wait until the table exists.
table.wait_until_exists()

# Print out some data about the table.
print(table.item_count)

0


In [5]:
# Get the table just created

table = dynamodb.Table('users')

print(table.creation_date_time)

2022-06-08 17:20:36.509000+00:00


In [6]:
#Creating a new item

table.put_item(
   Item={
        'username': 'janedoe',
        'first_name': 'Jane',
        'last_name': 'Doe',
        'age': 25,
        'account_type': 'standard_user',
    }
)

{'ResponseMetadata': {'RequestId': 'BJCMIRRASD190HPQ0V5P12HDEFVV4KQNSO5AEMVJF66Q9ASUAAJG',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'server': 'Server',
   'date': 'Wed, 08 Jun 2022 17:22:30 GMT',
   'content-type': 'application/x-amz-json-1.0',
   'content-length': '2',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'BJCMIRRASD190HPQ0V5P12HDEFVV4KQNSO5AEMVJF66Q9ASUAAJG',
   'x-amz-crc32': '2745614147'},
  'RetryAttempts': 0}}

In [7]:
#Getting an item

response = table.get_item(
    Key={
        'username': 'janedoe',
        'last_name': 'Doe'
    }
)
item = response['Item']
print(item)

{'username': 'janedoe', 'account_type': 'standard_user', 'last_name': 'Doe', 'first_name': 'Jane', 'age': Decimal('25')}


In [8]:
#Update item

table.update_item(
    Key={
        'username': 'janedoe',
        'last_name': 'Doe'
    },
    UpdateExpression='SET age = :val1',
    ExpressionAttributeValues={
        ':val1': 26
    }
)

{'ResponseMetadata': {'RequestId': '73HURIV06U0BJ1B0VOSI46345FVV4KQNSO5AEMVJF66Q9ASUAAJG',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'server': 'Server',
   'date': 'Wed, 08 Jun 2022 17:23:17 GMT',
   'content-type': 'application/x-amz-json-1.0',
   'content-length': '2',
   'connection': 'keep-alive',
   'x-amzn-requestid': '73HURIV06U0BJ1B0VOSI46345FVV4KQNSO5AEMVJF66Q9ASUAAJG',
   'x-amz-crc32': '2745614147'},
  'RetryAttempts': 0}}

In [10]:
#Getting the updated item

response = table.get_item(
    Key={
        'username': 'janedoe',
        'last_name': 'Doe'
    }
)
item = response['Item']
print(item)

{'username': 'janedoe', 'account_type': 'standard_user', 'last_name': 'Doe', 'age': Decimal('26'), 'first_name': 'Jane'}


In [11]:
#Deleting an item

table.delete_item(
    Key={
        'username': 'janedoe',
        'last_name': 'Doe'
    }
)

{'ResponseMetadata': {'RequestId': 'GD84HIAEA5DINGMVRDIKI5NVJBVV4KQNSO5AEMVJF66Q9ASUAAJG',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'server': 'Server',
   'date': 'Wed, 08 Jun 2022 17:24:21 GMT',
   'content-type': 'application/x-amz-json-1.0',
   'content-length': '2',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'GD84HIAEA5DINGMVRDIKI5NVJBVV4KQNSO5AEMVJF66Q9ASUAAJG',
   'x-amz-crc32': '2745614147'},
  'RetryAttempts': 0}}

In [12]:
#Batch Writing -> useful when we are writing lots of data

with table.batch_writer() as batch:
    batch.put_item(
        Item={
            'account_type': 'standard_user',
            'username': 'johndoe',
            'first_name': 'John',
            'last_name': 'Doe',
            'age': 25,
            'address': {
                'road': '1 Jefferson Street',
                'city': 'Los Angeles',
                'state': 'CA',
                'zipcode': 90001
            }
        }
    )
    batch.put_item(
        Item={
            'account_type': 'super_user',
            'username': 'janedoering',
            'first_name': 'Jane',
            'last_name': 'Doering',
            'age': 40,
            'address': {
                'road': '2 Washington Avenue',
                'city': 'Seattle',
                'state': 'WA',
                'zipcode': 98109
            }
        }
    )
    batch.put_item(
        Item={
            'account_type': 'standard_user',
            'username': 'bobsmith',
            'first_name': 'Bob',
            'last_name':  'Smith',
            'age': 18,
            'address': {
                'road': '3 Madison Lane',
                'city': 'Louisville',
                'state': 'KY',
                'zipcode': 40213
            }
        }
    )
    batch.put_item(
        Item={
            'account_type': 'super_user',
            'username': 'alicedoe',
            'first_name': 'Alice',
            'last_name': 'Doe',
            'age': 27,
            'address': {
                'road': '1 Jefferson Street',
                'city': 'Los Angeles',
                'state': 'CA',
                'zipcode': 90001
            }
        }
    )

In [13]:
#Batch writing a lot
with table.batch_writer() as batch:
    for i in range(50):
        batch.put_item(
            Item={
                'account_type': 'anonymous',
                'username': 'user' + str(i),
                'first_name': 'unknown',
                'last_name': 'unknown'
            }
        )

In [14]:
#QUERYING AND SCANNING
from boto3.dynamodb.conditions import Key, Attr
response = table.query(
    KeyConditionExpression=Key('username').eq('johndoe')
)
items = response['Items']
print(items)

[{'last_name': 'Doe', 'first_name': 'John', 'username': 'johndoe', 'address': {'zipcode': Decimal('90001'), 'state': 'CA', 'road': '1 Jefferson Street', 'city': 'Los Angeles'}, 'account_type': 'standard_user', 'age': Decimal('25')}]


In [15]:
response = table.scan(
    FilterExpression=Attr('age').lt(27)
)
items = response['Items']
print(items)

[{'last_name': 'Doe', 'first_name': 'John', 'username': 'johndoe', 'address': {'zipcode': Decimal('90001'), 'state': 'CA', 'road': '1 Jefferson Street', 'city': 'Los Angeles'}, 'account_type': 'standard_user', 'age': Decimal('25')}, {'last_name': 'Smith', 'first_name': 'Bob', 'username': 'bobsmith', 'address': {'zipcode': Decimal('40213'), 'state': 'KY', 'road': '3 Madison Lane', 'city': 'Louisville'}, 'account_type': 'standard_user', 'age': Decimal('18')}]


In [16]:
response = table.scan(
    FilterExpression=Attr('first_name').begins_with('J') & Attr('account_type').eq('super_user')
)
items = response['Items']
print(items)

[{'last_name': 'Doering', 'first_name': 'Jane', 'username': 'janedoering', 'address': {'zipcode': Decimal('98109'), 'state': 'WA', 'road': '2 Washington Avenue', 'city': 'Seattle'}, 'account_type': 'super_user', 'age': Decimal('40')}]


In [17]:
response = table.scan(
    FilterExpression=Attr('address.state').eq('CA')
)
items = response['Items']
print(items)

[{'last_name': 'Doe', 'first_name': 'John', 'username': 'johndoe', 'address': {'zipcode': Decimal('90001'), 'state': 'CA', 'road': '1 Jefferson Street', 'city': 'Los Angeles'}, 'account_type': 'standard_user', 'age': Decimal('25')}, {'last_name': 'Doe', 'first_name': 'Alice', 'username': 'alicedoe', 'address': {'zipcode': Decimal('90001'), 'state': 'CA', 'road': '1 Jefferson Street', 'city': 'Los Angeles'}, 'account_type': 'super_user', 'age': Decimal('27')}]


In [ ]:
#DELETING A TABLE
table.delete()